In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import cv2 
import os
from pathlib import Path
import json
%matplotlib inline

In [4]:
dir = '/root/capsule/data/behavior_711042_2024-09-13_09-19-15/behavior-videos/bottom_camera.avi'

In [5]:
def video_info_check(dir):
    base_name = os.path.splitext(os.path.basename(dir))[0]
    path_name = os.path.dirname(dir)
    video_info = dict()
    video_info['video_file'] = dir
    video_info['video_exist'] = os.path.exists(video_info['video_file'])
    video_info['timestamps_file'] = os.path.join(path_name, f"{base_name}.csv")
    video_info['timestamps_exist'] = os.path.exists(video_info['timestamps_file'])
    video_info['output_dir'] = os.path.join('/r oot/capsule/results', base_name)
    return video_info  

In [6]:
video_info = video_info_check(dir)
with open("video_qm.json", "w") as json_file:
    json.dump(video_info, json_file, indent=4, sort_keys=True)

In [7]:

def extract_frames(video_path, n_frames=1000):
    """
    Efficiently extracts frames from the video at regular intervals, storing them in a NumPy array.
    
    Args:
    - video_path (str): Path to the video file.
    - n_frames (int): Number of frames to extract.

    Returns:
    - frames (numpy array): Array containing the frames.
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open video.")
        return None

    # Get the total number of frames in the video
    total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval between frames to sample
    frame_interval = total_frame_count // n_frames if total_frame_count > n_frames else 1

    # Pre-allocate space for the frames
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to read first frame.")
        cap.release()
        return None
    
    # Get the frame dimensions (height, width, channels)
    height, width, channels = frame.shape
    frames = np.zeros((n_frames, height, width, channels), dtype=np.uint8)

    # Read frames at regular intervals and store them
    for i in range(n_frames):
        frame_number = i * frame_interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)  # Jump to the desired frame
        ret, frame = cap.read()
        if not ret:
            break
        frames[i] = frame

    # Release the video capture object
    cap.release()

    return frames

frames = extract_frames(video_info['video_file'], n_frames=1000)

In [14]:
def analyze_focus(frames, focus_threshold=60, output_plot='sharpness_distribution.png', output_example_frame='example_out_of_focus.png'):
    """
    Analyzes each frame to determine sharpness using Laplacian variance and outputs:
    - A sharpness distribution plot.
    - An example frame with the lowest sharpness (most out of focus).
    
    Args:
    - frames (list of numpy arrays): The frames to analyze.
    - focus_threshold (float): The variance threshold for focus detection.
    - output_plot (str): Path to save the sharpness distribution plot.
    - output_example_frame (str): Path to save the example frame with the lowest sharpness.
    
    Returns:
    - None: Saves the sharpness distribution plot and example frame to the specified locations.
    """
    sharpness_values = []  # List to store the Laplacian variance for each frame
    focus_results = []     # List to store focus status (True or False)
    lowest_sharpness_frame = None
    lowest_sharpness_value = float('inf')

    # Iterate through each frame and compute sharpness (Laplacian variance)
    for i, frame in enumerate(frames):
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Compute the Laplacian of the image and calculate the variance
        laplacian = cv2.Laplacian(gray_frame, cv2.CV_64F)
        focus_variance = laplacian.var()
        
        # Append the variance to the list of sharpness values
        sharpness_values.append(focus_variance)
        
        # Store the focus status (True if in focus, False if out of focus)
        is_focus = focus_variance > focus_threshold
        focus_results.append(is_focus)

        # Track the frame with the lowest sharpness (most out of focus)
        if focus_variance < lowest_sharpness_value:
            lowest_sharpness_value = focus_variance
            lowest_sharpness_frame = frame

    # Plot the sharpness distribution (Laplacian variance values)
    plt.hist(sharpness_values, bins=30, color='gray', edgecolor='black')
    plt.axvline(focus_threshold, color='red', linestyle='dashed', linewidth=2, label=f'Threshold ({focus_threshold})')
    plt.xlabel('Laplacian Variance (Sharpness)')
    plt.ylabel('Frequency')
    plt.title('Sharpness Distribution (Laplacian Variance)')
    plt.legend()
    plt.savefig(output_plot)
    plt.close()


    # Save the example frame with the lowest sharpness (most out of focus)
    if lowest_sharpness_frame is not None:
        cv2.imwrite(output_example_frame, lowest_sharpness_frame)

    # print results
    out_of_focus_count = len([x for x in focus_results if not x])
    total_frames = len(frames)
    out_of_focus_percentage = (out_of_focus_count / total_frames) * 100
    
    print(f"Total frames analyzed: {total_frames}")
    print(f"Frames out of focus: {out_of_focus_count} ({out_of_focus_percentage:.2f}%)")


analyze_focus(frames)


Total frames analyzed: 1000
Frames out of focus: 61 (6.10%)


In [11]:

def analyze_intensity(frames, lower_threshold=30, upper_threshold=40, output_csv='video_analysis.csv', plot_figure='pixel_distribution.png'):
    """
    Analyzes frames to calculate the mean pixel intensity and saves a histogram plot and the most under/overexposed frames.
    
    Args:
    - frames (numpy array): The frames to analyze.
    - lower_threshold (int): Minimum acceptable mean brightness (0-255).
    - upper_threshold (int): Maximum acceptable mean brightness (0-255).
    - output_csv (str): Path to save the CSV file with analysis results.
    - plot_figure (str): Path to save the histogram plot figure.
    
    Returns:
    - None: Saves analysis results to a CSV file and a histogram plot.
    """
    
    # List to store statistics for CSV
    statistics = []
    
    # Variables to track the underexposed and overexposed frames
    min_intensity = float('inf')
    max_intensity = -float('inf')
    min_frame = None
    max_frame = None

    # Iterate through each frame and analyze its brightness
    for i, frame in enumerate(frames):
        # Convert the frame to grayscale to simplify brightness analysis
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculate mean brightness of the frame
        mean_brightness = np.mean(gray_frame)

        # Update the min/max intensity and associated frames
        if mean_brightness < lower_threshold and mean_brightness < min_intensity:
            min_intensity = mean_brightness
            min_frame = frame  # Store the actual frame

        if mean_brightness > upper_threshold and mean_brightness > max_intensity:
            max_intensity = mean_brightness
            max_frame = frame  # Store the actual frame

        # Store the statistics for CSV
        statistics.append({
            'frame': i,
            'mean': mean_brightness,
            'underexposed': mean_brightness < lower_threshold,
            'overexposed': mean_brightness > upper_threshold
        })
    
    # Save the statistics to CSV
    df = pd.DataFrame(statistics)
    df.to_csv(output_csv, index=False)
    
    # Plot histogram of the mean intensities
    mean_intensities = [stat['mean'] for stat in statistics]
    plt.hist(mean_intensities, bins=30, color='gray', edgecolor='black')
    plt.axvline(lower_threshold, color='blue', linestyle='dashed', linewidth=2, label=f'Underexposure ({lower_threshold})')
    plt.axvline(upper_threshold, color='red', linestyle='dashed', linewidth=2, label=f'Overexposure ({upper_threshold})')
    plt.xlabel('Mean Intensity')
    plt.ylabel('Frequency')
    plt.title('Histogram of Frame Mean Intensities')
    plt.legend()
    
    # Save the histogram plot
    plt.savefig(plot_figure)
    plt.close()

    # Save the underexposed and overexposed images
    if min_frame is not None:
        cv2.imwrite('underexposed_frame.png', min_frame)
    if max_frame is not None:
        cv2.imwrite('overexposed_frame.png', max_frame)

    # Calculate counts for underexposed and overexposed frames
    underexposed_count = df['underexposed'].sum()
    overexposed_count = df['overexposed'].sum()
    total_frames = len(df)

    # Calculate percentages
    underexposed_percentage = (underexposed_count / total_frames) * 100
    overexposed_percentage = (overexposed_count / total_frames) * 100
    properly_exposed_count = total_frames - underexposed_count - overexposed_count
    properly_exposed_percentage = (properly_exposed_count / total_frames) * 100

    # Print summary with counts and percentages
    print(f"Total frames analyzed: {total_frames}")
    print(f"Underexposed frames: {underexposed_count} ({underexposed_percentage:.2f}%)")
    print(f"Overexposed frames: {overexposed_count} ({overexposed_percentage:.2f}%)")
    print(f"Properly exposed frames: {properly_exposed_count} ({properly_exposed_percentage:.2f}%)")


analyze_intensity(frames)


Total frames analyzed: 1000
Underexposed frames: 59 (5.90%)
Overexposed frames: 31 (3.10%)
Properly exposed frames: 910 (91.00%)


In [ ]:
 
def analyze_contrast(frames, lower_threshold=30, upper_threshold=40, output_csv='video_analysis.csv', plot_figure='pixel_distribution.png'):
    """
    Analyzes frames to calculate the mean pixel intensity and saves a histogram plot and the most under/overexposed frames.
    
    Args:
    - frames (numpy array): The frames to analyze.
    - lower_threshold (int): Minimum acceptable mean brightness (0-255).
    - upper_threshold (int): Maximum acceptable mean brightness (0-255).
    - output_csv (str): Path to save the CSV file with analysis results.
    - plot_figure (str): Path to save the histogram plot figure.
    
    Returns:
    - None: Saves analysis results to a CSV file and a histogram plot.
    """
    
    # List to store statistics for CSV
    statistics = []
    
    # Variables to track the underexposed and overexposed frames
    min_intensity = float('inf')
    max_intensity = -float('inf')
    min_frame = None
    max_frame = None

    # Iterate through each frame and analyze its brightness
    for i, frame in enumerate(frames):
        # Convert the frame to grayscale to simplify brightness analysis
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculate std of brightness of the frame
        contrast = np.std(gray_frame)

        # Update the min/max intensity and associated frames
        if mean_brightness < lower_threshold and mean_brightness < min_intensity:
            min_intensity = mean_brightness
            min_frame = frame  # Store the actual frame

        if mean_brightness > upper_threshold and mean_brightness > max_intensity:
            max_intensity = mean_brightness
            max_frame = frame  # Store the actual frame

        # Store the statistics for CSV
        statistics.append({
            'frame': i,
            'mean': mean_brightness,
            'underexposed': mean_brightness < lower_threshold,
            'overexposed': mean_brightness > upper_threshold
        })
    
    # Save the statistics to CSV
    df = pd.DataFrame(statistics)
    df.to_csv(output_csv, index=False)
    
    # Plot histogram of the mean intensities
    mean_intensities = [stat['mean'] for stat in statistics]
    plt.hist(mean_intensities, bins=30, color='gray', edgecolor='black')
    plt.axvline(lower_threshold, color='blue', linestyle='dashed', linewidth=2, label=f'Underexposure ({lower_threshold})')
    plt.axvline(upper_threshold, color='red', linestyle='dashed', linewidth=2, label=f'Overexposure ({upper_threshold})')
    plt.xlabel('Mean Intensity')
    plt.ylabel('Frequency')
    plt.title('Histogram of Frame Mean Intensities')
    plt.legend()
    
    # Save the histogram plot
    plt.savefig(plot_figure)
    plt.close()

    # Save the underexposed and overexposed images
    if min_frame is not None:
        cv2.imwrite('underexposed_frame.png', min_frame)
    if max_frame is not None:
        cv2.imwrite('overexposed_frame.png', max_frame)

    # Calculate counts for underexposed and overexposed frames
    underexposed_count = df['underexposed'].sum()
    overexposed_count = df['overexposed'].sum()
    total_frames = len(df)

    # Calculate percentages
    underexposed_percentage = (underexposed_count / total_frames) * 100
    overexposed_percentage = (overexposed_count / total_frames) * 100
    properly_exposed_count = total_frames - underexposed_count - overexposed_count
    properly_exposed_percentage = (properly_exposed_count / total_frames) * 100

    # Print summary with counts and percentages
    print(f"Total frames analyzed: {total_frames}")
    print(f"Underexposed frames: {underexposed_count} ({underexposed_percentage:.2f}%)")
    print(f"Overexposed frames: {overexposed_count} ({overexposed_percentage:.2f}%)")
    print(f"Properly exposed frames: {properly_exposed_count} ({properly_exposed_percentage:.2f}%)")


analyze_intensity(frames)

 
        